In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import csv

In [2]:
data=pd.read_csv("donnees_reddit_TS.csv")
data.head()

,suicide,subreddit,content
0,True,Suicidal_Thoughts,Suicidal_Thoughts has been created\r\nA subred...
1,True,Suicidal_Thoughts,Radio TTS prevented my suicide thoughts\r\nLat...
2,True,Suicidal_Thoughts,I really want to kill myself tonight. And I do...
3,True,Suicidal_Thoughts,"Nephew has suicidal thoughts, what to do?\r\nM..."
4,True,Suicidal_Thoughts,"I don�t want to die, I just want to cease to e..."


In [3]:
data.info

<bound method DataFrame.info of       suicide          subreddit  \
0        True  Suicidal_Thoughts   
1        True  Suicidal_Thoughts   
2        True  Suicidal_Thoughts   
3        True  Suicidal_Thoughts   
4        True  Suicidal_Thoughts   
...       ...                ...   
4718     True       SuicideWatch   
4719     True       SuicideWatch   
4720     True       SuicideWatch   
4721     True       SuicideWatch   
4722     True       SuicideWatch   

                                                content  
0     Suicidal_Thoughts has been created\r\nA subred...  
1     Radio TTS prevented my suicide thoughts\r\nLat...  
2     I really want to kill myself tonight. And I do...  
3     Nephew has suicidal thoughts, what to do?\r\nM...  
4     I don�t want to die, I just want to cease to e...  
...                                                 ...  
4718  isnt my life just that... MY life? so why is i...  
4719  What does SW think about the "I'm here to help...  
4720  Not qui

In [4]:
#change content en unicode
documents= data['content'].values.astype("U")

In [5]:
vectorizer= TfidfVectorizer(stop_words='english')
features =vectorizer.fit_transform(documents)

In [6]:
#determine le K qu'on va utiliser pour le clustering 
#3 cluster 
n_clusters=15
model=KMeans(n_clusters)
model.fit(features)

KMeans(n_clusters=15)

In [7]:
Y_pred = model.predict(features)

In [8]:
data['cluster']=model.labels_
data.head()

,suicide,subreddit,content,cluster
0,True,Suicidal_Thoughts,Suicidal_Thoughts has been created\r\nA subred...,1
1,True,Suicidal_Thoughts,Radio TTS prevented my suicide thoughts\r\nLat...,14
2,True,Suicidal_Thoughts,I really want to kill myself tonight. And I do...,9
3,True,Suicidal_Thoughts,"Nephew has suicidal thoughts, what to do?\r\nM...",3
4,True,Suicidal_Thoughts,"I don�t want to die, I just want to cease to e...",11


In [9]:
#create id 
data['Id']=data.index
data

,suicide,subreddit,content,cluster,Id
0,True,Suicidal_Thoughts,Suicidal_Thoughts has been created\r\nA subred...,1,0
1,True,Suicidal_Thoughts,Radio TTS prevented my suicide thoughts\r\nLat...,14,1
2,True,Suicidal_Thoughts,I really want to kill myself tonight. And I do...,9,2
3,True,Suicidal_Thoughts,"Nephew has suicidal thoughts, what to do?\r\nM...",3,3
4,True,Suicidal_Thoughts,"I don�t want to die, I just want to cease to e...",11,4
...,...,...,...,...,...
4718,True,SuicideWatch,isnt my life just that... MY life? so why is i...,9,4718
4719,True,SuicideWatch,"What does SW think about the ""I'm here to help...",2,4719
4720,True,SuicideWatch,Not quite suicide but I just made an extremely...,5,4720
4721,True,SuicideWatch,There is hope! I just want to share with you w...,5,4721


In [10]:
#group lines with same cluster 
clusters = data.groupby('cluster')

for cluster in clusters.groups: 
    f=open('cluster' + str(cluster)+'.csv', 'w') #create csv file
    data=clusters.get_group(cluster)[['subreddit','content']] #get subreddit and content columns 
    f.write(data.to_csv(index_label='Id')) #set index to id 
    f.close()

In [11]:
k=n_clusters
print('Cluster centroids: \n')
order_centroids= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(k):
    print('Cluster %d:' % i)
    for j in order_centroids[i, :10]: #print output 10 feature terms of each cluster
        print('%s' % terms[j])
    print('-------------')

Cluster centroids: 

Cluster 0:
lethal
dose
mg
does
just
tablets
want
die
painless
kill
-------------
Cluster 1:
life
just
like
people
fucking
ve
want
feel
hate
going
-------------
Cluster 2:
don
wanna
know
just
want
help
anymore
really
like
die
-------------
Cluster 3:
just
mom
didn
got
time
told
like
school
dad
went
-------------
Cluster 4:
im
just
dont
ive
want
feel
like
life
fucking
kill
-------------
Cluster 5:
ve
just
life
don
like
time
years
know
feel
going
-------------
Cluster 6:
suicidal
thoughts
ideation
just
like
don
feel
want
ve
know
-------------
Cluster 7:
don
want
just
life
people
know
feel
like
ve
think
-------------
Cluster 8:
dead
wish
just
don
life
want
think
alive
people
kill
-------------
Cluster 9:
kill
want
just
don
people
really
wish
try
life
tonight
-------------
Cluster 10:
prevention
hotline
suicide
called
national
chat
calling
help
crisis
just
-------------
Cluster 11:
feel
like
just
don
want
ve
know
really
life
feeling
-------------
Cluster 12:
dont
im
jus

In [ ]:
#from sklearn.metrics import completeness_score
#print(completeness_score(documents, Y_pred))